In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# train_path_img = "./yolo_data/images/train/" before
train_path_img = "/content/drive/MyDrive/ml_data/images/train/"
train_path_label = "/content/drive/MyDrive/ml_data/labels/train/"
val_path_img = "/content/drive/MyDrive/ml_data/images/val/"
val_path_label = "/content/drive/MyDrive/ml_data/labels/val/"
test_path = "/content/drive/MyDrive/ml_data/test"

In [ ]:

'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/ml_data/data/') ### without negative images

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data="/content/drive/MyDrive/ml_data/dataset.yaml" epochs=20 imgsz=1080 batch=8 project=/content

100% 21.5M/21.5M [00:00<00:00, 515MB/s]
Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/ml_data/dataset.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=1080, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fa

In [ ]:
!yolo task=detect mode=predict model="/content/train/weights/best.pt" conf=0.55 source="/content/drive/MyDrive/ml_data/test_images"

Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/17 /content/drive/MyDrive/ml_data/test_images/322.jpg: 640x1088 10 Players, 106.4ms
image 2/17 /content/drive/MyDrive/ml_data/test_images/328.jpg: 640x1088 10 Players, 25.6ms
image 3/17 /content/drive/MyDrive/ml_data/test_images/332.jpg: 640x1088 16 Players, 25.5ms
image 4/17 /content/drive/MyDrive/ml_data/test_images/335.jpg: 640x1088 1 Ball, 10 Players, 25.5ms
image 5/17 /content/drive/MyDrive/ml_data/test_images/339.jpg: 640x1088 13 Players, 25.5ms
image 6/17 /content/drive/MyDrive/ml_data/test_images/344.jpg: 640x1088 10 Players, 25.5ms
image 7/17 /content/drive/MyDrive/ml_data/test_images/364.jpg: 576x1088 14 Players, 83.2ms
image 8/17 /content/drive/MyDrive/ml_data/test_images/369.jpg: 640x1088 13 Players, 26.2ms
image 9/17 /content/drive/MyDrive/ml_data/test_images/372.jpg: 640x1088 18 Players, 25.5ms
ima

In [ ]:
rm /content/drive/MyDrive/ml_data/labels/val.cache

In [ ]:
!yolo task=detect mode=val model=yolov8s.pt data="/content/drive/MyDrive/ml_data/dataset_yolov8_notraining.yaml" imgsz=1080 batch=8

WARNING ⚠️ imgsz=[1080] must be multiple of max stride 32, updating to [1088]
Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs
val: Scanning /content/drive/MyDrive/ml_data/labels/val... 38 images, 0 backgrounds, 0 corrupt: 100% 38/38 [00:00<00:00, 132.09it/s]
val: New cache created: /content/drive/MyDrive/ml_data/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 5/5 [00:08<00:00,  1.76s/it]
                   all         38        579      0.504     0.0286   0.000869   0.000389
                person         30         35    0.00744     0.0571    0.00174   0.000778
               bicycle         38        544          1          0          0          0
Speed: 13.1ms preprocess, 45.3ms inference, 0.0ms loss, 30.7ms postprocess per image
Results saved to runs/detect/val7
💡 Learn more at https://docs.ultralyt

In [ ]:
!pip install streamlit torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00


In [ ]:
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
